In [1]:
import numpy as np
import datetime
import netCDF4
import gsw
#-------------------------------------------------------------------------|
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.colors as colors
import cmocean
import cmocean.cm as cmo
#-------------------------------------------------------------------------|
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from scipy.io import loadmat
import xarray as xr
import math
import h5py
import pandas as pd
import re
from pathlib import Path
import os
import numpy.ma as ma
import subprocess
import tqdm as tqdm
###############################
from pyresample import kd_tree, geometry, utils
from pyresample.geometry import GridDefinition
import pyproj as pyproj
from  pyproj import transform
from pyproj import Proj
# may make plotting faster
pyproj.set_use_global_context()
###############################
import sys
sys.path.append(r'/Users/houndegno/Documents/JupyterNoteBook/Prog/odi')
import odi
#-------------------------------------------------------------------------|
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# In situ CTD castaway data from SASSIE

In [2]:
#-------------------------------------------------------------------------------------------------------|
## SASSIE final data collected files directory ---------------------------------------------------------|
Final_SASSIE    = '/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/Final_SASSIE_update/'#------------|
## Data files directory --------------------------------------------------------------------------------|
CastAway_CTD    = 'CastAway_CTD/'#----------------------------------------------------------------------|
#-------------------------------------------------------------------------------------------------------|
#Saving_Folder = '/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/Final_SASSIE_Coloc/UpDate_colocs/'#-|
#-------------------------------------------------------------------------------------------------------|

In [3]:
## Cast away CTD data list
CastAway_CTD_content = os.listdir(Final_SASSIE+CastAway_CTD)
CastAway_CTD_content

['SASSIE_Fall_2022_SHIPBOARD_Castaway_CTD.nc']

In [4]:
CTD_cast = xr.open_dataset(Final_SASSIE+CastAway_CTD+CastAway_CTD_content[0])
CTD_cast

<xarray.Dataset>
Dimensions:      (time: 250, depth: 311)
Coordinates:
  * time         (time) datetime64[ns] 2022-09-09T15:22:59.999998208 ... 2022...
  * depth        (depth) float64 1.0 1.296 1.591 1.887 ... 92.06 92.35 92.65
    longitude    (time) float64 ...
    latitude     (time) float64 ...
Data variables:
    pressure     (depth, time) float64 ...
    temperature  (depth, time) float64 ...
    salinity     (depth, time) float64 ...
    density      (depth, time) float64 ...
Attributes: (12/54)
    DOI:                           10.5067/SASSIE-RVCTD2
    title:                         SASSIE Arctic Field Campaign Castaway Data...
    summary:                       Castaway data collected from R/V Woldstad ...
    keywords:                      EARTH SCIENCE > OCEANS > OCEAN TEMPERATURE...
    keywords_vocabulary:           NASA Global Change Master Directory (GCMD)...
    conventions:                   CF-1.8, ACDD-1.4
    ...                            ...
    geospatial_lon_max:            -145.3295382
    geospatial_lon_units:          degrees_east
    time_coverage_start:           2022-09-09T15:23:00Z
    time_coverage_end:             2022-09-19T00:27:00Z
    uuid:                          ac6f60ba-010d-472a-8fab-57b0f8483d87
    time_coverage_duration:        P09DT09H04M00S

In [5]:
# longitude & latitude 
Lon_CTD = np.array(CTD_cast.longitude)
Lat_CTD = np.array(CTD_cast.latitude)
# time related to each profile
Time_CTD = CTD_cast.time

In [6]:
Lon_CTD[0]

-149.96155675

pd.to_datetime(str(np.array(Time_CTD[0]))[slice(0,10,1)]).dayofyear

# Atmospheric forcing data from GES DISC

In [7]:
rep = '/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/MITgcm_experiments/1D_OJH/ForcingData/'
M2I1NXASM = 'M2I1NXASM.5.12.4.nc'
M2T1NXFLX = 'M2T1NXFLX.5.12.4.nc'
M2T1NXRAD = 'M2T1NXRAD.5.12.4.nc'

# M2I1NXASM.5.12.4
Available vraiables:
* 2-meter specific humidity above ocean (QV2M)
* sea level pressure (SLP)
* 2-meter air temperature above ocean (T2M)
* 10-meter eastward wind above ocean (U10M)
* 10-meter northward wind above ocean (V10M)
* Time (time)
* Longitude (lat)
* Latitude (lon)

In [8]:
ds = xr.open_dataset(rep+M2I1NXASM)
ds

<xarray.Dataset>
Dimensions:  (time: 3672, lat: 25, lon: 46)
Coordinates:
  * lat      (lat) float64 68.0 68.5 69.0 69.5 70.0 ... 78.0 78.5 79.0 79.5 80.0
  * lon      (lon) float64 -168.1 -167.5 -166.9 -166.2 ... -141.2 -140.6 -140.0
  * time     (time) datetime64[ns] 2022-09-01 ... 2023-01-31T23:00:00
Data variables:
    T2M      (time, lat, lon) float32 ...
    SLP      (time, lat, lon) float32 ...
    V10M     (time, lat, lon) float32 ...
    U10M     (time, lat, lon) float32 ...
    QV2M     (time, lat, lon) float32 ...
Attributes: (12/32)
    History:                           Original file generated: Tue Sep 13 03...
    Comment:                           GMAO filename: d5124_m2_jan10.inst1_2d...
    Filename:                          MERRA2_400.inst1_2d_asm_Nx.20220901.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    RangeBeginningDate:                2022-09-01
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2022-09-01
    RangeEndingTime:                   23:00:00.000000
    DODS_EXTRA.Unlimited_Dimension:    time
    history:                           2023-10-24 04:26:55 GMT Hyrax-1.16.3 h...

In [9]:
# Direct extraction of data
lon  = ds.lon
lat  = ds.lat
Time = ds.time
# daily mean apply
T2M  = ds.T2M.groupby("time.dayofyear").mean(dim='time')
SLP  = ds.SLP.groupby("time.dayofyear").mean(dim='time')
U10M = ds.U10M.groupby("time.dayofyear").mean(dim='time')
V10M = ds.V10M.groupby("time.dayofyear").mean(dim='time')
QV2M = ds.QV2M.groupby("time.dayofyear").mean(dim='time')

ForcingTime = T2M.dayofyear
ForcingTime

In [10]:
# time arangement with dayofyear
time = ds.time.dt.dayofyear
atime = pd.unique(time)
# time reconstruction to fit the applied daily mean (time at 12AM)
indtime = np.arange(atime.size)
for i in np.arange(atime.size):
    indtime[i] = np.where(time == atime[i])[0][0]
# Good time to use
Time_go = Time[indtime]
Time_go
# re-organizing variable to fit time unique order
reorg_indtime = np.arange(atime.size)
for i in np.arange(atime.size):
    reorg_indtime[i] = np.where(QV2M["dayofyear"] == atime[i])[0]
# variable re-arangement
T2M  = T2M[reorg_indtime,:,:]
SLP  = SLP[reorg_indtime,:,:]
U10M = U10M[reorg_indtime,:,:]
V10M = V10M[reorg_indtime,:,:]
QV2M = QV2M[reorg_indtime,:,:]

# M2T1NXFLX.5.12.4
Available vraiables:
* EVAP: evaporation by turbulence
* PRECTOT: total_precipitation
* PRECTOTCORR: bias_corrected_total_precipitation
* TAUX: eastward_stress
* TAUY: northward_stress
* I add Ice covered fraction
* Time (time)
* Longitude (lat)
* Latitude (lon)

In [11]:
ds = xr.open_dataset(rep+M2T1NXFLX)
ds

<xarray.Dataset>
Dimensions:      (time: 3672, lat: 33, lon: 46)
Coordinates:
  * lat          (lat) float64 64.0 64.5 65.0 65.5 66.0 ... 78.5 79.0 79.5 80.0
  * lon          (lon) float64 -168.1 -167.5 -166.9 ... -141.2 -140.6 -140.0
  * time         (time) datetime64[ns] 2022-09-01T00:30:00 ... 2023-01-31T23:...
Data variables:
    PRECTOT      (time, lat, lon) float32 ...
    EVAP         (time, lat, lon) float32 ...
    TAUX         (time, lat, lon) float32 ...
    PRECTOTCORR  (time, lat, lon) float32 ...
    TAUY         (time, lat, lon) float32 ...
    FRSEAICE     (time, lat, lon) float32 ...
Attributes: (12/32)
    History:                           Original file generated: Tue Sep 13 02...
    Comment:                           GMAO filename: d5124_m2_jan10.tavg1_2d...
    Filename:                          MERRA2_400.tavg1_2d_flx_Nx.20220901.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    RangeBeginningDate:                2022-09-01
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2022-09-01
    RangeEndingTime:                   23:59:59.000000
    DODS_EXTRA.Unlimited_Dimension:    time
    history:                           2023-10-24 17:33:46 GMT Hyrax-1.16.3 h...

In [12]:
Lon2 = ds.lon
Lat2 = ds.lat
Time2 = ds.time
# daily mean apply
EVAP = ds.EVAP.groupby("time.dayofyear").mean(dim='time')
PrecipTot = ds.PRECTOT.groupby("time.dayofyear").mean(dim='time')
PrecipCorr = ds.PRECTOTCORR.groupby("time.dayofyear").mean(dim='time')
Taux = ds.TAUX.groupby("time.dayofyear").mean(dim='time')
Tauy = ds.TAUY.groupby("time.dayofyear").mean(dim='time')
IceFrac = ds.FRSEAICE.groupby("time.dayofyear").mean(dim='time')
# variable re-arangement
EVAP       = EVAP[reorg_indtime,:,:]
PrecipTot  = PrecipTot[reorg_indtime,:,:]/1000 # converting from kg m-2 s-1 to m3.m-2.s-1
PrecipCorr = PrecipCorr[reorg_indtime,:,:]/1000 # converting from kg m-2 s-1 to m3.m-2.s-1
Taux       = Taux[reorg_indtime,:,:]
Tauy       = Tauy[reorg_indtime,:,:]
IceFrac    = IceFrac[reorg_indtime,:,:]

# M2T1NXRAD.5.12.4
Available vraiables:
* LWGAB: surface absorbed longwave radiation
* SWGDN: surface incoming shortwave flux
* Time (time)
* Longitude (lat)
* Latitude (lon)

In [13]:
ds = xr.open_dataset(rep+M2T1NXRAD)
ds

<xarray.Dataset>
Dimensions:  (time: 3672, lat: 33, lon: 46)
Coordinates:
  * lat      (lat) float64 64.0 64.5 65.0 65.5 66.0 ... 78.0 78.5 79.0 79.5 80.0
  * lon      (lon) float64 -168.1 -167.5 -166.9 -166.2 ... -141.2 -140.6 -140.0
  * time     (time) datetime64[ns] 2022-09-01T00:30:00 ... 2023-01-31T23:30:00
Data variables:
    LWGAB    (time, lat, lon) float32 ...
    SWGDN    (time, lat, lon) float32 ...
Attributes: (12/32)
    History:                           Original file generated: Tue Sep 13 02...
    Comment:                           GMAO filename: d5124_m2_jan10.tavg1_2d...
    Filename:                          MERRA2_400.tavg1_2d_rad_Nx.20220901.nc4
    Conventions:                       CF-1
    Institution:                       NASA Global Modeling and Assimilation ...
    References:                        http://gmao.gsfc.nasa.gov
    ...                                ...
    RangeBeginningDate:                2022-09-01
    RangeBeginningTime:                00:00:00.000000
    RangeEndingDate:                   2022-09-01
    RangeEndingTime:                   23:59:59.000000
    DODS_EXTRA.Unlimited_Dimension:    time
    history:                           2023-10-24 17:54:18 GMT Hyrax-1.16.3 h...

In [14]:
lon3 = ds.lon
lat3 = ds.lat
LWGAB = ds.LWGAB.groupby("time.dayofyear").mean(dim='time')
SWGDN = ds.SWGDN.groupby("time.dayofyear").mean(dim='time')
# variable re-arangement
LWGAB = LWGAB[reorg_indtime,:,:]
SWGDN = SWGDN[reorg_indtime,:,:]

# Selection of forcing data matching in situ data CTD profile

In [15]:
Global_RUN = "/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/MITgcm_experiments/1D_OJH/1D_ocean_ice_column/Global_RUN2/"
Generic = "/Users/houndegno/Documents/MATLAB/Data/SASSIE_DATA/MITgcm_experiments/1D_OJH/1D_ocean_ice_column/Generic/"
Generic_Content = os.listdir(Generic)
Generic_Content

['eedata',
 'data.diagnostics_old',
 'data.pkg',
 'generic_zero_1x1',
 'data.diagnostics',
 'mitgcmuv',
 'data.cal',
 'tr_checklist',
 'data.exf',
 'data.err',
 'STDERR.0000',
 'generic_one_1x1',
 'bathy_1x1_1105m_testpool',
 'data.mnc',
 'data',
 'data.seaice',
 'snow_0m_1x1',
 'data.kpp']

In [16]:
for i in tqdm.tqdm(np.arange(Lon_CTD.size)):
    #------------------------------------------------------------#
    # in situ profile corrdinate
    lonB  = Lon_CTD[i]
    latB  = Lat_CTD[i]
    timeB = Time_CTD[i]
    #------------------------------------------------------------#
    # setting lon and lat str() format --------------------------#
    a = np.array(lonB).round(2)
    b = np.array(latB).round(2)
    if a>0:
        lonstr = str(a)+'E'
    else:
        lonstr = str(np.abs(a))+'W'
    if b>0:
        latstr = str(b)+'N'
    else:
        latstr = str(np.abs(a))+'S'
    # date str() format
    ts_date = str(np.array(timeB))[slice(0,10,1)]
    # Surfix of file name
    Surfix_name = "_1x1_four_months_Lon_"+lonstr+"_Lat_"+latstr+".bin"
    #------------------------------------------------------------#
    # Creating the proper "run" folder
    #savemeto = Global_RUN+"run_"+ts_date+"_"+str(i+1).zfill(3)+'/'
    savemeto = Global_RUN+"run_"+str(i+1).zfill(3)+'/'
    os.mkdir(savemeto)
    #------------------------------------------------------------#
    # copying the generic files into the proper "run" folder
    for ij in np.arange(len(Generic_Content)):
        subprocess.call(['cp', Generic+Generic_Content[ij],savemeto])
    #------------------------------------------------------------#
    # selection of in situ associated atmospheric forcing data
    t2m = T2M.sel(lon=lonB,lat=latB,method='nearest')
    slp = SLP.sel(lon=lonB,lat=latB,method='nearest')
    u10m = U10M.sel(lon=lonB,lat=latB,method='nearest')
    v10m = V10M.sel(lon=lonB,lat=latB,method='nearest')
    qv2m = QV2M.sel(lon=lonB,lat=latB,method='nearest')
    evap = EVAP.sel(lon=lonB,lat=latB,method='nearest')
    precipcorr = PrecipCorr.sel(lon=lonB,lat=latB,method='nearest')
    taux = Taux.sel(lon=lonB,lat=latB,method='nearest')
    tauy = Tauy.sel(lon=lonB,lat=latB,method='nearest')
    icefrac = IceFrac.sel(lon=lonB,lat=latB,method='nearest')
    lwgab = LWGAB.sel(lon=lonB,lat=latB,method='nearest')
    swgdn = SWGDN.sel(lon=lonB,lat=latB,method='nearest')
    #------------------------------------------------------------#
    # Saving ".bin" files into the proper "run" directory
    # air temperature at 2m above ocean
    np.array(t2m).astype('>f4').tofile(savemeto+'atemp'+Surfix_name)
    # sea level pressure
    np.array(slp).astype('>f4').tofile(savemeto+'slp'+Surfix_name)
    # wind u and v at 10m above ocean
    np.array(u10m).astype('>f4').tofile(savemeto+'u_10m'+Surfix_name)
    np.array(v10m).astype('>f4').tofile(savemeto+'v_10m'+Surfix_name)
    # easterward and northwar wind stress
    np.array(taux).astype('>f4').tofile(savemeto+'taux'+Surfix_name)
    np.array(tauy).astype('>f4').tofile(savemeto+'tauy'+Surfix_name)
    # specific humidity at 2m above ocean
    np.array(qv2m).astype('>f4').tofile(savemeto+'qv2m'+Surfix_name)
    # evaporation
    np.array(evap).astype('>f4').tofile(savemeto+'evap'+Surfix_name)
    # total precipitation
    np.array(precipcorr).astype('>f4').tofile(savemeto+'precipcorr'+Surfix_name)
    # surface absorbed longwave radiation
    np.array(lwgab).astype('>f4').tofile(savemeto+'lwgab'+Surfix_name)
    # surface incoming shortwave_flux
    np.array(swgdn).astype('>f4').tofile(savemeto+'swgdn'+Surfix_name)

100%|████████████████████████████████████████████████████| 250/250 [05:23<00:00,  1.29s/it]


# Saving the IceFraction from observation

In [17]:
for i in np.arange(Lon_CTD.size):
    #------------------------------------------------------------#
    # in situ profile corrdinate
    lonB  = Lon_CTD[i]
    latB  = Lat_CTD[i]
    #------------------------------------------------------------#
    #savemeto = Global_RUN+"run_"+ts_date+"_"+str(i+1).zfill(3)+'/'
    savemeto = Global_RUN+"run_"+str(i+1).zfill(3)+'/'
    #------------------------------------------------------------#
    # selection of in situ associated atmospheric forcing data
    icefrac = IceFrac.sel(lon=lonB,lat=latB,method='nearest')
    #------------------------------------------------------------#
    # Saving ".bin" files into the proper "run" directory
    # surface incoming shortwave_flux
    np.array(icefrac).astype('>f4').tofile(savemeto+'IceFrac_four_months.bin')

2635000/(3600*24)